<a href="https://colab.research.google.com/github/syedmahmoodiagents/Basics/blob/main/Langchain_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install langchain==1.2.1

  Using cached langchain-1.2.1-py3-none-any.whl.metadata (4.9 kB)
Using cached langchain-1.2.1-py3-none-any.whl (105 kB)


In [ ]:
import langchain, langchain_core

In [ ]:
langchain.__version__

'1.2.0'

In [ ]:
!pip install langchain-huggingface --q

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

In [ ]:
import os, getpass

In [ ]:
os.environ['HF_TOKEN'] = getpass.getpass()

··········


In [ ]:
llmhg = ChatHuggingFace(llm=HuggingFaceEndpoint(repo_id='meta-llama/Llama-3.1-8B-Instruct'))

In [ ]:
res = llmhg.invoke('What is the capital of France')

In [ ]:
res.content

'The capital of France is Paris.'

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory

In [ ]:
mem = InMemoryChatMessageHistory()

In [ ]:
mem.add_user_message('What is the capital of France')

In [ ]:
mem.add_ai_message('Paris')

In [ ]:
mem.add_user_message('What is the capital of Italy')

In [ ]:
mem

InMemoryChatMessageHistory(messages=[HumanMessage(content='What is the capital of France', additional_kwargs={}, response_metadata={}), AIMessage(content='Paris', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of Italy', additional_kwargs={}, response_metadata={})])

In [ ]:
mem.messages[-1].content

'What is the capital of Italy'

In [ ]:
mem.messages

[HumanMessage(content='What is the capital of France', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Paris', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the capital of Italy', additional_kwargs={}, response_metadata={})]

In [ ]:
llmhg.invoke(mem.messages)

AIMessage(content='Rome', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 58, 'total_tokens': 61}, 'model_name': 'meta-llama/Llama-3.1-8B-Instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019ba10f-1ca2-7d00-8ce1-4a4b00e12441-0', usage_metadata={'input_tokens': 58, 'output_tokens': 3, 'total_tokens': 61})

In [ ]:
llmhg.invoke(mem.messages[-1].content)

AIMessage(content='The capital of Italy is Rome.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 41, 'total_tokens': 49}, 'model_name': 'meta-llama/Llama-3.1-8B-Instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b9c80-3232-7430-912b-d91554dcd448-0', usage_metadata={'input_tokens': 41, 'output_tokens': 8, 'total_tokens': 49})

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    # For simplicity, we are still using the global 'mem' object for all sessions.
    # In a real application, you would manage separate histories per session_id.
    return mem

# Define a prompt template that includes a placeholder for history
# and a placeholder for the current human message.
prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="history"), # This placeholder will be filled by RunnableWithMessageHistory
        ("human", "{question}"), # This is the placeholder for the current user input
    ]
)

# Chain the prompt with the language model.
# The prompt will generate a list of messages that llmhg can process.
chain_with_history_handling = prompt | llmhg

with_message_history = RunnableWithMessageHistory(
    chain_with_history_handling, # Now wrapping the chain, not just the LLM directly
    get_session_history,
    input_messages_key="question", # This corresponds to '{question}' in the prompt
    history_messages_key="history" # This corresponds to 'MessagesPlaceholder(variable_name="history")'
)

# Example of invoking with a session
# The input to invoke should be a dictionary with the key specified by input_messages_key
res_with_history = with_message_history.invoke(
    {"question": "What is the capital of Japan?"}, # Pass the current message content
    config={"configurable": {"session_id": "my_session"}}
)

print(res_with_history.content)

# To demonstrate history working, let's ask a follow-up question in the same session
res_with_history_followup = with_message_history.invoke(
    {"question": "And what is its population?"},
    config={"configurable": {"session_id": "my_session"}}
)

print(res_with_history_followup.content)

Tokyo
The population of Tokyo is approximately 39.1 million people (as of 2021), with its metropolitan area having a population of over 38 million people.
